In [115]:
import random
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import PDSVRP_instance
from importnb import Notebook
import copy

with Notebook():
    import heuristic as h
    import utilities as u

In [129]:
# Parameter ranges
param_ranges = {
    'num_customers': (10, 100),
    'grid_size': (20, 40),
    'num_drones': (1, 20),
    'battery_endurance': (0.25, 1.0),  # in hours
    'cost_ratio': (10.0, 50.0)
}

# Constants
constants = {
    'truck_speed': 30.0,
    'drone_speed': 40.0,
    'truck_capacity': 1300.0,
    'drone_capacity': 2.27,
    'drone_unit_cost': 0.03,
    'work_time_limit': 8.0
}

num_instances = 10

In [130]:
def generate_instance():
    grid_size = random.choice(param_ranges['grid_size'])
    custumers = []
    
    depot = [{
        'x': grid_size/2,
        'y': grid_size/2, 
        'demand': 0.00
    }]
    
    custumers = [
            {
                'x': random.uniform(0, param_ranges['grid_size'][1]),
                'y': random.uniform(0, param_ranges['grid_size'][1]),
                'demand': random.uniform(0, 3) if np.random.rand() < 0.6 else random.uniform(5, 50) 
            }
            for _ in range(random.randint(*param_ranges['num_customers']))
        ]
    
    all_entities  = depot + custumers
    
    header = (
        f"NUM DRONES,{random.randint(*param_ranges['num_drones'])}\n"
        f"NUM TRUCKS,{10}\n"
        f"TRUCK CAP,{constants['truck_capacity']}\n"
        f"DRONE CAP,{constants['drone_capacity']}\n"
        f"TRUCK SPEED,{constants['truck_speed']}\n"
        f"DRONE SPEED,{constants['drone_speed']}\n"
        f"DRONE ENDURANCE,{random.uniform(*param_ranges['battery_endurance']):.2f}\n"
        f"DRONE TIME LIMIT,{random.uniform(1.0, constants['work_time_limit']):.2f}\n"
        f"TRUCK TIME LIMIT,{random.uniform(1.0, constants['work_time_limit']):.2f}\n"
        f"TRUCK UNIT COST,{(random.uniform(*param_ranges['cost_ratio']) * constants['drone_unit_cost']):.2f}\n"
        f"DRONE UNIT COST,{constants['drone_unit_cost']}\n"
    )
    customer_lines = "\n".join(
        f"{i}\t{customer['x']:.2f}\t{customer['y']:.2f}\t{customer['demand']:.2f}"
        for i, customer in enumerate(all_entities)
    )
    return header + customer_lines

In [131]:
# Generate all instances
instances = [generate_instance()  for _ in range(num_instances)]

In [132]:
for index, instance in enumerate(instances):
    with open('./generated_instances/'+ str(index) + '-r-r' '.txt', 'w') as file:
        file.write(str(instance))

In [139]:
def compute_cost_saving(instance_name, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    instance = PDSVRP_instance.PDSVRPInstance("generated_instances/" + instance_name)
    
    instance_no_drones = copy.deepcopy(instance)
    instance_no_drones.D = 0
    
    sol_drones = h.SISSRs(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
    sol_no_drones = h.SISSRs(instance_no_drones, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb)
    
    cost_with_drones = u.cost(instance, sol_drones)
    cost_without_drones = u.cost(instance_no_drones, sol_no_drones)
    
    cost_saving = 100 * (cost_without_drones - cost_with_drones) / cost_without_drones
    return cost_saving

In [140]:
# Apply the function to each instance
cost_savings = []

instances_name_files = []
for indee in range(num_instances):
    instances_name_files.append(str(index) + "-r-r.txt")
        
for name_file_instance in instances_name_files:
    cost_savings.append(compute_cost_saving(name_file_instance, 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 1000, 10000, 3, 3, 9))

Iteration:  1
Costo miglior soluzione:  91.24581218256175
Iteration:  2
Costo miglior soluzione:  91.24581218256175
Iteration:  3
Costo miglior soluzione:  91.24581218256175
Iteration:  4
Costo miglior soluzione:  91.24581218256175
Iteration:  5
Costo miglior soluzione:  91.24581218256175
Iteration:  6
Costo miglior soluzione:  90.00501218256173
Iteration:  7
Costo miglior soluzione:  90.00501218256173
Iteration:  8
Costo miglior soluzione:  90.00501218256173
Iteration:  9
Costo miglior soluzione:  90.00501218256173


KeyboardInterrupt: 